In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image
import cv2
from matplotlib.widgets import Button

# Problem 1.1

In [ ]:
def create_gaussian_kernel(k, s):
    """
    Create a 2D Gaussian kernel.
    """
    ax = np.arange(-k // 2 + 1., k // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * s**2))
    return kernel / np.sum(kernel)
def apply_convolution(image, kernel):
    """
    Apply convolution to the image using the given kernel.
    """
    m, n = kernel.shape
    y, x = image.shape
    y = y - m + 1
    x = x - n + 1
    output = np.zeros((y,x))
    for i in range(y):
        for j in range(x):
            output[i,j] = np.sum(image[i:i+m, j:j+n]*kernel)
    return output
def gradient_edge_detector(image, sigma):
    """
    Perform gradient-based edge detection as per PDF instructions.
    """
    # Create Gaussian kernel
    kernel_size = int(6 * sigma + 1)  # Ensure kernel size is odd
    if kernel_size % 2 == 0:
        kernel_size += 1
    gaussian_kernel = create_gaussian_kernel(kernel_size, sigma)
    
    # Smooth the image with a Gaussian filter
    smoothed = apply_convolution(image, gaussian_kernel)
    
    # Compute horizontal and vertical derivatives
    dx_kernel = np.array([[-1, 0, 1]])
    dy_kernel = np.array([[-1], [0], [1]])
    
    grad_x = apply_convolution(smoothed, dx_kernel)
    grad_y = apply_convolution(smoothed, dy_kernel)
    
    # Ensure grad_x and grad_y have the same shape
    min_shape = (min(grad_x.shape[0], grad_y.shape[0]), min(grad_x.shape[1], grad_y.shape[1]))
    grad_x = grad_x[:min_shape[0], :min_shape[1]]
    grad_y = grad_y[:min_shape[0], :min_shape[1]]
    
    # Compute gradient magnitude and orientation
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    orientation = np.arctan2(grad_y, grad_x)
    
    return magnitude, orientation
def plot_results(image, magnitude, orientation):
    """
    Plot the original image, gradient magnitude, and gradient orientation.
    """
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(magnitude, cmap='gray')
    plt.title('Gradient Magnitude')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    step = 10  # Adjust this to change the density of arrows
    y, x = np.mgrid[step//2:magnitude.shape[0]:step, step//2:magnitude.shape[1]:step]
    u = np.cos(orientation[y, x])
    v = np.sin(orientation[y, x])
    plt.quiver(x, y, u, v, color='r', angles='xy', scale_units='xy', scale=0.1)
    plt.title('Gradient Orientation')
    plt.gca().invert_yaxis()
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
#Load and preprocess the image
image = np.array(Image.open('./1.png').convert('L'))
image = image.astype(np.float32) / 255.0  # Normalize to [0, 1]

# Set parameter
sigma = 1.0

# Apply gradient-based edge detection
magnitude, orientation = gradient_edge_detector(image, sigma)
# Plot results
plot_results(image, magnitude, orientation)
# You can experiment with different sigma values here
# sigmas = [0.5, 1.0, 2.0]
# for sigma in sigmas:
#     magnitude, orientation = gradient_edge_detector(image, sigma)
#     plot_results(image, magnitude, orientation)
#     print(f"Results for sigma = {sigma}")

# Problem 1.2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import matplotlib
from PIL import Image
matplotlib.use('macosx')

class BoundaryTracer:
    def __init__(self, image_path):
        # Load and preprocess the image
        self.original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        self.edges = cv2.Canny(self.original_image, 100, 200)
        self.edges = cv2.dilate(self.edges, np.ones((3,3), np.uint8), iterations=1)
        
        # Store clicked points and traced boundaries
        self.clicked_points = []
        self.boundaries = []
        
        # Setup the plot
        self.fig, self.ax = plt.subplots(figsize=(10, 8))
        self.setup_plot()
        
    def setup_plot(self):
        """Initialize the plot with the original image"""
        self.ax.clear()
        self.ax.imshow(self.original_image, cmap='gray')
        self.ax.set_title('Click on object boundaries to trace them\nPress Enter when done')
        
        # Plot existing boundaries
        for boundary in self.boundaries:
            if len(boundary):
                self.ax.plot(boundary[:, 0], boundary[:, 1], 'r-', linewidth=2)
        
        # Plot clicked points
        for point in self.clicked_points:
            self.ax.plot(point[0], point[1], 'go', markersize=8)
            
        self.fig.canvas.draw()
    
    def trace_boundary(self, start_x, start_y, max_points=1000):
        """
        Trace the boundary starting from a given point using 8-connectivity
        Returns array of boundary points
        """
        # Check if starting point is on an edge
        if self.edges[start_y, start_x] == 0:
            return np.array([])
            
        boundary_points = [(start_x, start_y)]
        visited = np.zeros_like(self.edges, dtype=bool)
        visited[start_y, start_x] = True
        
        # 8-connected neighborhood coordinates
        neighbors = [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]
        
        current_x, current_y = start_x, start_y
        
        while len(boundary_points) < max_points:
            found_next = False
            
            # Check all neighbors in clockwise order
            for dx, dy in neighbors:
                next_x, next_y = current_x + dx, current_y + dy
                
                # Check bounds
                if (0 <= next_x < self.edges.shape[1] and 
                    0 <= next_y < self.edges.shape[0] and 
                    self.edges[next_y, next_x] > 0 and 
                    not visited[next_y, next_x]):
                    
                    boundary_points.append((next_x, next_y))
                    visited[next_y, next_x] = True
                    current_x, current_y = next_x, next_y
                    found_next = True
                    break
            
            if not found_next:
                break
                
        return np.array(boundary_points)
    
    def onclick(self, event):
        """Handle mouse clicks"""
        if event.inaxes != self.ax:
            return
            
        x, y = int(event.xdata), int(event.ydata)
        self.clicked_points.append((x, y))
        
        # Trace boundary from clicked point
        boundary = self.trace_boundary(x, y)
        if len(boundary) > 0:
            self.boundaries.append(boundary)
        
        # Redraw the plot
        self.setup_plot()
    
    def onkey(self, event):
        """Handle key presses"""
        if event.key == 'enter':
            plt.close(self.fig)
    
    def run(self):
        """Run the interactive boundary tracing"""
        self.fig.canvas.mpl_connect('button_press_event', self.onclick)
        self.fig.canvas.mpl_connect('key_press_event', self.onkey)
        plt.show()
        
        # After closing, show final result
        if self.boundaries:
            self.show_final_result()
    
    def show_final_result(self):
        """Display the final result with all traced boundaries"""
        plt.figure(figsize=(12, 6))
        
        # Original image with boundaries
        plt.subplot(121)
        plt.imshow(self.original_image, cmap='gray')
        plt.title('Original Image with Traced Boundaries')
        for boundary in self.boundaries:
            if len(boundary):
                plt.plot(boundary[:, 0], boundary[:, 1], 'r-', linewidth=2)
        plt.axis('off')
        
        # Edge map with boundaries
        plt.subplot(122)
        plt.imshow(self.edges, cmap='gray')
        plt.title('Edge Map with Traced Boundaries')
        for boundary in self.boundaries:
            if len(boundary):
                plt.plot(boundary[:, 0], boundary[:, 1], 'r-', linewidth=2)
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()

# Usage example
tracer = BoundaryTracer('./1.png')
tracer.run()

# Problem 2

In [ ]:
# Assume gradient_edge_detector, create_gaussian_kernel, and apply_convolution are already defined

def compute_gradients(image):
    """Compute x and y gradients of the image."""
    dx = np.zeros(image.shape)
    dy = np.zeros(image.shape)
    
    dx[:, 1:-1] = image[:, 2:] - image[:, :-2]
    dy[1:-1, :] = image[2:, :] - image[:-2, :]
    
    return dx, dy

def harris_corner_detector(image, k=0.04, window_size=3, threshold=0.01):
    """
    Implement Harris corner detector.
    
    Args:
    image: 2D numpy array of the input image
    k: Harris detector free parameter
    window_size: Size of the window for local maximum suppression
    threshold: Threshold for corner response
    
    Returns:
    corner_response: Corner response for each pixel
    corners: Binary image with detected corners
    """
    dx, dy = compute_gradients(image)
    
    gaussian_kernel = create_gaussian_kernel(7, 1)  # Adjust size and sigma as needed
    dx2 = apply_convolution(dx**2, gaussian_kernel)
    dy2 = apply_convolution(dy**2, gaussian_kernel)
    dxy = apply_convolution(dx*dy, gaussian_kernel)
    
    # Compute the Harris corner response
    det = dx2 * dy2 - dxy**2
    trace = dx2 + dy2
    corner_response = det - k * trace**2
    
    # Threshold and local maximum suppression
    corners = np.zeros_like(corner_response, dtype=bool)
    for i in range(window_size//2, corner_response.shape[0] - window_size//2):
        for j in range(window_size//2, corner_response.shape[1] - window_size//2):
            window = corner_response[i-window_size//2:i+window_size//2+1,
                                     j-window_size//2:j+window_size//2+1]
            if corner_response[i, j] == np.max(window) and corner_response[i, j] > threshold:
                corners[i, j] = True
    
    return corner_response, corners

def plot_corners(image, corners):
    """Plot the original image with detected corners."""
    plt.figure(figsize=(12, 6))
    
    plt.subplot(121)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(122)
    plt.imshow(image, cmap='gray')
    y, x = np.where(corners)
    plt.scatter(x, y, c='red', s=5)
    plt.title('Detected Corners')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Load and preprocess the image
image_path = './2-1.jpg'  # Replace with your image path
image = np.array(Image.open(image_path).convert('L'))
image = image.astype(np.float32) / 255.0  # Normalize to [0, 1]

# Apply Harris corner detector
corner_response, corners = harris_corner_detector(image)

# Plot results
plot_corners(image, corners)

# Print number of detected corners
print(f"Number of detected corners: {np.sum(corners)}")

# Problem 3.1

In [ ]:
def edge_detection(image, low_threshold=0.1, high_threshold=0.2):
    """
    Perform edge detection using the gradient method and apply double thresholding.
    """
    magnitude, _ = gradient_edge_detector(image, sigma=1.0)
    low_mask = magnitude > low_threshold
    high_mask = magnitude > high_threshold
    edge_map = ((low_mask & high_mask) | high_mask).astype(np.uint8)
    return edge_map

def myHoughLine(imBW, n, rho_res=1, theta_res=0.5):
    """
    Implement Hough transform to detect line segments in a binary image.
    """
    height, width = imBW.shape
    diag_len = int(np.ceil(np.sqrt(height**2 + width**2)))
    rhos = np.arange(-diag_len, diag_len, rho_res)
    thetas = np.deg2rad(np.arange(-90, 90, theta_res))
    
    cos_t = np.cos(thetas)
    sin_t = np.sin(thetas)
    
    accumulator = np.zeros((len(rhos), len(thetas)))
    
    y_idxs, x_idxs = np.nonzero(imBW)
    
    for i, j in zip(x_idxs, y_idxs):
        for theta_idx in range(len(thetas)):
            rho = int((i * cos_t[theta_idx] + j * sin_t[theta_idx]) / rho_res) + diag_len
            if 0 <= rho < len(rhos):
                accumulator[rho, theta_idx] += 1
    
    lines = []
    for _ in range(n):
        idx = np.argmax(accumulator)
        rho_idx, theta_idx = np.unravel_index(idx, accumulator.shape)
        rho = rhos[rho_idx]
        theta = thetas[theta_idx]
        lines.append((rho, theta))
        accumulator[max(0, rho_idx-5):min(accumulator.shape[0], rho_idx+6),
                    max(0, theta_idx-5):min(accumulator.shape[1], theta_idx+6)] = 0
    
    return lines

def find_line_segments(image, lines, min_length=20):
    """
    Find line segments in the image based on Hough lines.
    """
    segments = []
    for rho, theta in lines:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        
        # Create a mask for the line
        mask = np.zeros_like(image)
        cv2.line(mask, (x1, y1), (x2, y2), 1, 1)
        
        # Find where the line intersects with edges
        intersection = np.logical_and(mask, image)
        points = np.argwhere(intersection)
        
        if len(points) >= 2:
            dist = np.linalg.norm(points[-1] - points[0])
            if dist >= min_length:
                segments.append((tuple(points[0][::-1]), tuple(points[-1][::-1])))
    
    return segments

def plot_line_segments(image, segments):
    """
    Plot the original image with detected line segments.
    """
    plt.figure(figsize=(12, 6))
    
    plt.subplot(121)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(122)
    plt.imshow(image, cmap='gray')
    plt.title('Detected Line Segments')
    plt.axis('off')
    
    for (x1, y1), (x2, y2) in segments:
        plt.plot([x1, x2], [y1, y2], 'r-')
    
    plt.tight_layout()
    plt.show()

# Load and preprocess the image
image_path = './3.png'  # Replace with your image path
image = np.array(Image.open(image_path).convert('L'))
image = image.astype(np.float32) / 255.0  # Normalize to [0, 1]

# Apply edge detection
edge_image = edge_detection(image, low_threshold=0.01, high_threshold=0.03)

# Detect lines using Hough transform
n_lines = 5  # Increased number of lines to detect
detected_lines = myHoughLine(edge_image, n_lines, rho_res=1, theta_res=0.5)

# Find line segments
line_segments = find_line_segments(edge_image, detected_lines, min_length=20)

# Plot results
plot_line_segments(image, line_segments)

print(f"Detected {len(line_segments)} line segments:")
for i, ((x1, y1), (x2, y2)) in enumerate(line_segments, 1):
    print(f"Segment {i}: ({x1}, {y1}) to ({x2}, {y2})")

# Problem 3.2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def myHoughCircleTrain(imBW, c, ptlist):
    """
    Train the Hough transform for circle detection.
    
    Args:
    imBW: Binary input image containing a single circular object
    c: Reference point (center of the circle)
    ptlist: Ordered list of boundary points
    
    Returns:
    yourcellvar: Cell array containing necessary data for circle detection
    """
    # Calculate radius
    radii = [np.sqrt((x - c[0])**2 + (y - c[1])**2) for x, y in ptlist]
    radius = np.mean(radii)
    
    # Create a template for circle detection
    theta = np.linspace(0, 2*np.pi, 100)
    x_template = radius * np.cos(theta)
    y_template = radius * np.sin(theta)
    
    # Store the template and radius in a cell array
    yourcellvar = {
        'radius': radius,
        'x_template': x_template,
        'y_template': y_template
    }
    
    return yourcellvar

def myHoughCircleTest(imBWnew, yourcellvar):
    """
    Test the Hough transform for circle detection on a new image.
    
    Args:
    imBWnew: New binary input image to detect circles in
    yourcellvar: Cell array containing data from training
    
    Returns:
    centers: List of detected circle centers (top 2)
    """
    radius = yourcellvar['radius']
    x_template = yourcellvar['x_template']
    y_template = yourcellvar['y_template']
    
    height, width = imBWnew.shape
    accumulator = np.zeros((height, width))
    
    # Perform Hough transform
    edge_points = np.argwhere(imBWnew > 0)
    for y, x in edge_points:
        for dx, dy in zip(x_template, y_template):
            a = int(x - dx)
            b = int(y - dy)
            if 0 <= a < width and 0 <= b < height:
                accumulator[b, a] += 1
    
    # Find the top 2 peaks in the accumulator
    centers = []
    for _ in range(2):
        idx = np.unravel_index(np.argmax(accumulator), accumulator.shape)
        centers.append((idx[1], idx[0]))  # (x, y) format
        # Suppress this maximum and its neighborhood
        y, x = idx
        y_min, y_max = max(0, y - 10), min(height, y + 11)
        x_min, x_max = max(0, x - 10), min(width, x + 11)
        accumulator[y_min:y_max, x_min:x_max] = 0
    
    return centers

def plot_results(image, centers, radius):
    """
    Plot the original image with detected circles.
    """
    plt.figure(figsize=(10, 5))
    
    plt.subplot(121)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(122)
    plt.imshow(image, cmap='gray')
    plt.title('Detected Circles')
    plt.axis('off')
    
    for center in centers:
        circle = plt.Circle(center, radius, fill=False, color='r')
        plt.gca().add_artist(circle)
    
    plt.tight_layout()
    plt.show()

# Example usage
# Load and preprocess the training image
train_image_path = './train.png'  # Replace with your training image path
train_image = np.array(Image.open(train_image_path).convert('L'))
train_binary = (train_image > 128).astype(np.uint8)

# Assume we have the center and boundary points for the training image
center = (100, 100)  # Example center, replace with actual center
boundary_points = [(110, 100), (100, 110), (90, 100), (100, 90)]  # Example points, replace with actual boundary points

# Train the circle detector
trained_data = myHoughCircleTrain(train_binary, center, boundary_points)

# Load and preprocess the test image
test_image_path = './test.png'  # Replace with your test image path
test_image = np.array(Image.open(test_image_path).convert('L'))
test_binary = (test_image > 128).astype(np.uint8)

# Detect circles in the test image
detected_centers = myHoughCircleTest(test_binary, trained_data)

# Plot results
plot_results(test_image, detected_centers, trained_data['radius'])

print("Detected circle centers:")
for i, center in enumerate(detected_centers, 1):
    print(f"Circle {i}: center at {center}")